In [68]:
# Auto PEP8 Code
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [69]:
# Enable Plots in Jupyter Notebook
%matplotlib inline

<IPython.core.display.Javascript object>

In [70]:
from skimage import feature
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys


<IPython.core.display.Javascript object>

In [ ]:
# Test HoG with sample image where square is oriented almost straight
# Original Image
image = cv2.imread("./input/shapes/square/0.png")
plt.imshow(image)
plt.title("Original Image")
plt.show()

# Run HoGs on image
(hog, hog_image) = feature.hog(
    image,
    orientations=9,
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2),
    block_norm="L2",
    visualize=True,
    transform_sqrt=True,
)

plt.imshow(hog_image)
plt.title("HoGs Image")
plt.show()

In [ ]:
# Pyramid
import imutils
import cv2


def pyramid(image, scale=2, minSize=(10, 10)):
    # yield the original image
    yield image
    # keep looping over the pyramid
    while True:
        # compute the new dimensions of the image and resize it
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)
        # if the resized image does not meet the supplied minimum
        # size, then stop constructing the pyramid
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
        # yield the next image in the pyramid
        yield image


# construct the argument parser and parse the arguments
image = cv2.imread("./test_images/shapes/square-1.png")

# METHOD #1: No smooth, just scaling.
# loop over the image pyramid
for (i, resized) in enumerate(pyramid(image)):
    # show the resized image
    plt.imshow(resized)
    plt.show()

In [182]:
# Complete Code
import os

import cv2
import imutils
from sklearn.svm import LinearSVC
from skimage import feature
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

<IPython.core.display.Javascript object>

In [190]:
### Training Set
# Get HOG Features from Training Set
img_path = "shapes"
img_path = f"input/{img_path}"

x_train = []
y_train = []

# Get all the image folder paths
img_paths = os.listdir(img_path)
for path in img_paths:

    # Get lists containing names of all images for training set
    main_path = f"{img_path}/{path}"
    all_images = os.listdir(main_path)

    # Run HoGs on training set
    for image in all_images:

        # Get image
        image_path = f"{main_path}/{image}"

        # Read image and resize to 64 x 128 - HoGs requires 1:2 ratio
        image = cv2.imread(image_path)
        image = cv2.resize(image, (64, 128))

        # Calculate HOG descriptor for each image
        hog_desc = feature.hog(
            image,
            orientations=9,
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2),
            transform_sqrt=True,
            block_norm="L2",
        )

        # Add images and labels
        x_train.append(hog_desc)
        y_train.append(path)

<IPython.core.display.Javascript object>

In [191]:
### Train SVM
print("Training SVM on the training set ...")
svm_model = LinearSVC(random_state=42, tol=1e-5)
svm_model.fit(x_train, y_train)

Training SVM on the training set ...


LinearSVC(random_state=42, tol=1e-05)

<IPython.core.display.Javascript object>

In [197]:
def sliding_window(image, step_sz, window_sz):
    for y in range(0, image.shape[0], step_sz):
        for x in range(0, image.shape[1], step_sz):
            # yield the current window
            yield (x, y, image[y : y + window_sz[1], x : x + window_sz[0]])


def pyramid(image, scale=2.5, minSize=(5, 5)):
    # yield the original image
    yield image
    # keep looping over the pyramid
    while True:
        # compute the new dimensions of the image and resize it
        w = int(image.shape[1] / scale)
        #         image = imutils.resize(image, width=w)
        width = int(image.shape[1] / scale)
        height = int(image.shape[0] / scale)
        dim = (width, height)
        # resize image
        image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
        # if the resized image does not meet the supplied minimum
        # size, then stop constructing the pyramid
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
        # yield the next image in the pyramid
        yield image


# image = cv2.imread("./test_images/shapes/square-1.png")
# loop over the image pyramid
# for (i, resized) in enumerate(pyramid(image)):
#     # show the resized image
#     plt.imshow(resized)
#     plt.show()


def test_set_algo(image, svm_model, step_sz=8, window_sz=(100, 100)):
    x_test = []
    y_test = []
    image_meta = []
    image = cv2.resize(image, (200, 200))
    win_width, win_height = window_sz
    for (i, image_pyr) in enumerate(pyramid(image)):
        for x, y, window in sliding_window(
            image_pyr, step_sz=step_sz, window_sz=window_sz
        ):
            # Ignore if window does not match shape
            if window.shape[0] != win_height or window.shape[1] != win_width:
                continue

            # Resize to HoG size - 1:2 ratio
            window = cv2.resize(window, (64, 128))
            (hog_desc, hog_image) = feature.hog(
                window,
                orientations=9,
                pixels_per_cell=(8, 8),
                cells_per_block=(2, 2),
                block_norm="L2",
                visualize=True,
                transform_sqrt=True,
            )

            x_pred = hog_desc.reshape(1, -1)
            y_pred = svm_model.predict(x_pred)[0]

            x_test.append(x_pred)
            y_test.append(y_pred)
            image_meta.append((x, y, hog_image))

    # Get idx of best fitting point
    des_func = list(map(svm_model.decision_function, x_test))
    des_max = max(des_func)
    des_min = min(des_func)
    
    if abs(des_max) > abs(des_min):
        des_val = des_max
    else:
        des_val = des_min
        
    print(des_val)
    des_func_idx = des_func.index(des_val)
    label = y_test[des_func_idx]

    # Get best fitting sliding window data
    x, y, hog_image = image_meta[des_func_idx]

    # Draw bounding box on image
    image = cv2.rectangle(
        image, (x, y), (x + win_width, y + win_height), (0, 255, 0), 2
    )

        plt.clf()
        plt.imshow(image)
        plt.show()
        print(label)
    return label, image, hog_image


image = cv2.imread("./test_images/shapes/square-35.png")
label, image, hog_image = test_set_algo(image, svm_model)

[-1.89815629]


<IPython.core.display.Javascript object>

In [198]:
# Run SVM model on test images
print("Testing SVM model on testing set ...")

img_path = "stars"
img_path = f"test_images/{img_path}"
img_paths = os.listdir(img_path)

output_path = f"outputs/"

x_pred_all = []
y_pred_all = []

# loop over the test dataset folders
for (i, img) in enumerate(img_paths):
    # File pathing
    img = f"{img_path}/{img}"
    image = cv2.imread(img)

    # Run HoGs algorithim and sliding window
    label, image, hog_image = test_set_algo(image, svm_model)

    # Print predictions
    img_name = img.split("/")[-1][:-4]

    print("Actual >>>", img_name)
    print("Predicted >>>", label)
    print("\n")

    # Rescale HoG image
    hog_image = hog_image.astype("float64")

    # Add text to image
    actual_text = f"Actual: {img_name}"
    predicted_text = f"Predicted: {label}"

    cv2.putText(
        image, predicted_text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2
    )
    cv2.putText(
        image, actual_text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2
    )

    # Write output images to output folder
    cv2.imwrite(f"{output_path}hog_{i}.jpg", hog_image * 255.0)
    cv2.imwrite(f"{output_path}pred_{i}.jpg", image)

Testing SVM model on testing set ...
[-1.53830575]
Actual >>> square-3
Predicted >>> square


[-0.49519798]
Actual >>> square-4
Predicted >>> square


[-1.11390476]
Actual >>> square-5
Predicted >>> square


[-1.07162649]
Actual >>> square-6
Predicted >>> square


[-0.5206366]
Actual >>> square-7
Predicted >>> square


[-1.05299597]
Actual >>> Star_fcfc7886-2a87-11ea-8123-8363a7ec19e6
Predicted >>> square


[-1.44292708]
Actual >>> Star_fd0202ac-2a95-11ea-8123-8363a7ec19e6
Predicted >>> square


[-0.99302803]
Actual >>> Star_fd03cd2e-2a83-11ea-8123-8363a7ec19e6
Predicted >>> square


[-1.66248084]
Actual >>> Star_fd1c056c-2a86-11ea-8123-8363a7ec19e6
Predicted >>> square


[-0.43095213]
Actual >>> Star_fd1c4550-2a8a-11ea-8123-8363a7ec19e6
Predicted >>> square


[-0.58472702]
Actual >>> Star_fd2f1c20-2a8f-11ea-8123-8363a7ec19e6
Predicted >>> square


[-1.53237497]
Actual >>> Star_fd36b39c-2a97-11ea-8123-8363a7ec19e6
Predicted >>> square


[-1.61944492]
Actual >>> Star_fd3b1f72-2a92-11ea-

<IPython.core.display.Javascript object>